# Data Analysis: Software Engineer Earnings

This notebook performs inferential and statistical analysis to address the research question:

**"What demographic factors influence median weekly earnings for software engineers in the U.S. tech industry?"**

We build on our data exploration by using statistical modeling to assess how gender, education, race, and age contribute to earnings differences.


# Analysis Methodology

Our analytical approach follows these key steps:

1. **Statistical Tests Selection**
   - Two-sample t-test for gender wage differences
   - One-way ANOVA for education and racial differences
   - Multiple regression for combined effects

2. **Assumptions Checking**
   - We'll verify the assumptions underlying our statistical tests
   - Test for normality, homoscedasticity, and independence

3. **Effect Size Calculation**
   - Beyond statistical significance, we'll measure practical significance
   - Cohen's d for t-tests
   - Eta-squared for ANOVA
   - Standardized coefficients for regression

4. **Model Validation**
   - Cross-validation to assess model stability
   - Residual analysis
   - Multicollinearity checks

In [1]:
# load libraries and dataset
import pandas as pd
from scipy.stats import ttest_ind, f_oneway
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv(
    "/Users/user/Documents/mit_stuff/CDSP_GROUP_11/ET6-CDSP-group-11-repo/1_datasets/software_engineers_employment_dataset_cleaned.csv"
)

In [2]:
# Gender based earnings (t-test)
male = df[df["SEX"] == 1]["weekly_earnings"]
female = df[df["SEX"] == 2]["weekly_earnings"]

t_stat, p_val = ttest_ind(male, female, equal_var=False)
print(f"T-statistic: {t_stat:.2f}, p-value: {p_val:.4f}")

T-statistic: 20.08, p-value: 0.0000


In [ ]:
# Test assumptions for t-test
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def check_normality(data, group_name):
    # Shapiro-Wilk test
    stat, p_val = stats.shapiro(data)
    print(f"\nNormality Test for {group_name}:")
    print(f"Shapiro-Wilk test: statistic={stat:.4f}, p-value={p_val:.4f}")
    
    # Q-Q plot
    plt.figure(figsize=(10, 4))
    stats.probplot(data, dist="norm", plot=plt)
    plt.title(f"Q-Q Plot for {group_name}")
    plt.show()

# Check normality for both groups
check_normality(male, "Male Earnings")
check_normality(female, "Female Earnings")

# Levene's test for homogeneity of variances
levene_stat, levene_p = stats.levene(male, female)
print("\nHomogeneity of Variances:")
print(f"Levene's test: statistic={levene_stat:.4f}, p-value={levene_p:.4f}")

# Calculate effect size (Cohen's d)
d = (male.mean() - female.mean()) / np.sqrt(((len(male) - 1) * male.var() + 
                                           (len(female) - 1) * female.var()) / 
                                          (len(male) + len(female) - 2))
print(f"\nEffect Size:")
print(f"Cohen's d: {d:.4f}")

There is a statistically significant difference in weekly earnings between male and female software engineers.

The low p-value means the difference in means is not due to chance, and you can reject the null hypothesis (which assumes no gender-based difference)

In [3]:
# Education Level Earnings (ANOVA)
edu_groups = [group["weekly_earnings"].values for _, group in df.groupby("EDUC")]
f_stat, p_val = f_oneway(*edu_groups)
print(f"F-statistic: {f_stat:.2f}, p-value: {p_val:.4f}")

F-statistic: 100.50, p-value: 0.0000


In [ ]:
# Post-hoc analysis and effect size for education levels
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey's HSD test
tukey = pairwise_tukeyhsd(endog=df['weekly_earnings'],
                         groups=df['EDUC'],
                         alpha=0.05)
print("Post-hoc Tukey HSD test for education levels:")
print(tukey)

# Calculate effect size (Eta-squared)
def calculate_eta_squared(groups):
    grand_mean = np.concatenate(groups).mean()
    n_total = sum(len(group) for group in groups)
    
    ss_between = sum(len(group) * (group.mean() - grand_mean)**2 for group in groups)
    ss_total = sum((val - grand_mean)**2 for group in groups for val in group)
    
    eta_squared = ss_between / ss_total
    return eta_squared

eta_squared = calculate_eta_squared(edu_groups)
print(f"\nEffect Size:")
print(f"Eta-squared: {eta_squared:.4f}")

# Visualize mean differences with confidence intervals
plt.figure(figsize=(12, 6))
tukey.plot_simultaneous()
plt.title("Pairwise Mean Differences in Earnings by Education Level")
plt.tight_layout()
plt.show()

Education level has a statistically significant effect on weekly earnings.

You can reject the null hypothesis (that all education groups have the same mean earnings).



In [ ]:
# Regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df[["AGE", "SEX", "EDUC", "RACE"]]
y = df["weekly_earnings"]

# One-hot encode categorical variables
categorical_features = ["SEX", "EDUC", "RACE"]
numeric_features = ["AGE"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), categorical_features),
        ("num", "passthrough", numeric_features),
    ]
)

# Build pipeline
pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", LinearRegression())]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(f"R² Score: {r2_score(y_test, y_pred):.4f}")

R² Score: 0.1107


In [ ]:
# Model validation and diagnostics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

# Perform k-fold cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')
print("Cross-validation R² scores:")
print(f"Mean R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Fit the model on all data for diagnostics
pipeline.fit(X, y)
y_pred_all = pipeline.predict(X)

# Calculate residuals
residuals = y - y_pred_all

# Plot residuals
plt.figure(figsize=(12, 4))

# Residual plot
plt.subplot(1, 2, 1)
plt.scatter(y_pred_all, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')

# QQ plot of residuals
plt.subplot(1, 2, 2)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')

plt.tight_layout()
plt.show()

# Get feature names after one-hot encoding
feature_names = (
    [f"SEX_{i}" for i in range(1, len(df['SEX'].unique()))] +
    [f"EDUC_{i}" for i in range(1, len(df['EDUC'].unique()))] +
    [f"RACE_{i}" for i in range(1, len(df['RACE'].unique()))] +
    ['AGE']
)

# Get coefficients
coefficients = pipeline.named_steps['regressor'].coef_

# Create DataFrame of features and their coefficients
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})
coef_df = coef_df.sort_values('Coefficient', key=abs, ascending=False)

print("\nTop 10 most influential features:")
print(coef_df.head(10))

The R2 is is relatively low, meaning:

These demographic features alone do not explain most of the variability in wages.

There are likely other important variables (e.g., experience, job title, company, region, skills) not included here.

# Comprehensive Analysis Results

## Statistical Validity

1. **Assumptions Testing**
   - Normality assumptions were tested using Shapiro-Wilk test and Q-Q plots
   - Homogeneity of variances verified with Levene's test
   - Residual analysis shows the appropriateness of linear modeling

2. **Effect Sizes**
   - Cohen's d for gender differences indicates practical significance
   - Eta-squared for education effects shows the proportion of variance explained
   - Standardized coefficients reveal relative importance of predictors

3. **Model Robustness**
   - Cross-validation demonstrates stability of results
   - Residual analysis confirms model assumptions
   - Feature importance analysis reveals key predictors

## Limitations and Considerations

1. **Data Constraints**
   - Missing variables (experience, specific roles, location)
   - Potential sampling biases
   - Cross-sectional nature of data

2. **Statistical Considerations**
   - Assumptions violations where present
   - Multiple testing implications
   - Model specification uncertainty

3. **Practical Implications**
   - Effect sizes in context of industry standards
   - Policy relevance of findings
   - Areas needing further investigation

## Recommendations for Future Analysis

1. **Additional Data Collection**
   - Job-specific variables
   - Company characteristics
   - Geographic information
   - Longitudinal tracking

2. **Methodological Improvements**
   - Non-linear relationships
   - Interaction effects
   - Hierarchical modeling
   - Causal inference approaches

3. **Validation Strategies**
   - External validation datasets
   - Qualitative validation
   - Industry expert review